In [1]:
import pandas as pd
import numpy as np
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm 
from spacy.util import minibatch, compounding
from spacy import displacy
import string
from spacy.gold import GoldParse
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
import pickle
import warnings
warnings.filterwarnings('ignore')

## Data preprocessing

In [2]:
df = pd.read_csv("data.csv")

In [ ]:
def obtain_annotation(title):
    
    # Initial annotation, charactors' ranges are highly overlapped
    entity_set_list=[]
    entity_dict={}
    entity_set_range_list=[]
    for m in range(0,len(entity)):
        if entity[m] in title:
            index_i=title.find(entity[m])
            index_f=index_i+len(entity[m])
            if (index_i != 0) and (index_f != len(title)):
                if (title[index_i-1] == ' ' and title[index_f] == ' '):
                    #print(entity[m])
                    entity_tuple=(index_i, index_f, entity_types[m])
                    entity_set_list.append(entity_tuple)
                    entity_set_range_list.append(range(index_i, index_f))
            if (index_i == 0) and (index_f != len(title)):
                if (title[index_f] == ' '):
                    #print(entity[m])
                    entity_tuple=(index_i, index_f, entity_types[m])
                    entity_set_list.append(entity_tuple)
                    entity_set_range_list.append(range(index_i, index_f))
            if (index_i != 0) and (index_f == len(title)):
                if (title[index_i-1] == ' '):
                    #print(entity[m])
                    entity_tuple=(index_i, index_f, entity_types[m])
                    entity_set_list.append(entity_tuple)
                    entity_set_range_list.append(range(index_i, index_f))

    # Second Step: Get rid of overlapped charactors' ranges
    entity_set_list_2=[]
    for n, entity_set_range_1 in enumerate(entity_set_range_list):
        entity_set_range_test=set(entity_set_range_1)
        inter=0
        entity_set_list_2_temp=[]
        for m, entity_set_range_2 in enumerate(entity_set_range_list):
            if entity_set_range_1 != entity_set_range_2:
                interss=entity_set_range_test.intersection(entity_set_range_2)
                #entity_set_list_2_temp=[]
                if interss==set():
                    inter += 1
                else:
                    if set(entity_set_range_1)>set(entity_set_range_2):
                        if entity_set_list[n] not in entity_set_list_2_temp:
                            entity_set_list_2_temp.append(entity_set_list[n])
                    elif set(entity_set_range_1)<set(entity_set_range_2):
                        if entity_set_list[m] not in entity_set_list_2_temp:
                            entity_set_list_2_temp.append(entity_set_list[m])
        
        if m == inter:
            if entity_set_list[n] not in entity_set_list_2:
                entity_set_list_2.append(entity_set_list[n])
        else:
            for entity_set_list_2_temp_item in entity_set_list_2_temp:
                if entity_set_list_2_temp_item not in entity_set_list_2:
                    entity_set_list_2.append(entity_set_list_2_temp_item)
    
    # Third Step: Get rid of overlapped charactors' ranges further if any
    entity_set_list_3=[]
    for n, item_1 in enumerate(entity_set_list_2):
        item_1_range=range(item_1[0],item_1[1])
        inter=0
        entity_set_list_2_temp=[]
        for m, item_2 in enumerate(entity_set_list_2):
            item_2_range=range(item_2[0],item_2[1])
            if item_1_range != item_2_range:
                interss=set(item_1_range).intersection(item_2_range)
                if interss == set():
                    inter += 1
                else:
                    if set(item_1_range)>set(item_2_range):
                        if item_1 not in entity_set_list_2_temp:
                            entity_set_list_2_temp.append(item_1)
                    elif set(item_1_range)<set(item_2_range):
                        if item_2 not in entity_set_list_2_temp:
                            entity_set_list_2_temp.append(item_2)
        if m == inter:
            entity_set_list_3.append(item_1)
        else:
            for entity_set_list_2_temp_item in entity_set_list_2_temp:
                if entity_set_list_2_temp_item not in entity_set_list_3:
                    entity_set_list_3.append(entity_set_list_2_temp_item)
    
    # Fourth Step: Get rid of overlapped charactors' ranges further if any
    entity_set_list_4=[]
    for n, item_1 in enumerate(entity_set_list_3):
        item_1_range=range(item_1[0],item_1[1])
        inter=0
        entity_set_list_2_temp=[]
        for m, item_2 in enumerate(entity_set_list_3):
            item_2_range=range(item_2[0],item_2[1])
            if item_1_range != item_2_range:
                interss=set(item_1_range).intersection(item_2_range)
                if interss == set():
                    inter += 1
                else:
                    if set(item_1_range)>set(item_2_range):
                        if item_1 not in entity_set_list_2_temp:
                            entity_set_list_2_temp.append(item_1)
                    elif set(item_1_range)<set(item_2_range):
                        if item_2 not in entity_set_list_2_temp:
                            entity_set_list_2_temp.append(item_2)
        if m == inter:
            entity_set_list_4.append(item_1)
        else:
            for entity_set_list_2_temp_item in entity_set_list_2_temp:
                if entity_set_list_2_temp_item not in entity_set_list_4:
                    entity_set_list_4.append(entity_set_list_2_temp_item)
    
    # Construct annotation
    entity_dict['entities']=entity_set_list_4
    annotation_n=(title, entity_dict)
    return annotation_n

In [ ]:
TRAIN_DATA_0=[]
for n in range(0,len(titles_unique_brand)):
    annotation=obtain_annotation(titles_unique_brand[n])
    if annotation[1]['entities']!=[]:
        TRAIN_DATA_0.append(annotation)

In [ ]:
TRAIN_DATA_1=[]
for n in range(0,len(description_unique_brand)):
    annotation=obtain_annotation(description_unique_brand[n])
    if annotation[1]['entities']!=[]:
        TRAIN_DATA_1.append(annotation)
TRAIN_DATA = TRAIN_DATA_0 + TRAIN_DATA_1

## Annotated Data

In [3]:
with open("Ner_Pickle.txt", "rb") as fp:   
    DATA = pickle.load(fp)
TRAIN_DATA=DATA[0:100]
TEST_DATA=DATA[101:200]

## Build Model

In [4]:
def built_model(data,iterations,drop=0.2):
    TRAIN_DATA = data
    nlp = spacy.blank('xx')  
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes): 
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  
                    [annotations],  
                    drop=drop,  
                    sgd=optimizer,  
                    losses=losses)
            print(losses)
    return nlp

## Train Model

In [5]:
%%time
nlp = built_model(TRAIN_DATA,10)

Statring iteration 0
{'ner': 454.16092656680775}
Statring iteration 1
{'ner': 348.53016700584993}
Statring iteration 2
{'ner': 335.6208784650256}
Statring iteration 3
{'ner': 225.72551010572485}
Statring iteration 4
{'ner': 298.4570892778641}
Statring iteration 5
{'ner': 181.81861411529565}
Statring iteration 6
{'ner': 213.67335806500316}
Statring iteration 7
{'ner': 194.15879235694246}
Statring iteration 8
{'ner': 148.21225148669745}
Statring iteration 9
{'ner': 109.74437068745043}
CPU times: user 56.9 s, sys: 968 ms, total: 57.9 s
Wall time: 29.9 s


In [6]:
pickle.dump(nlp, open('model.pkl','wb'))

## Evaluate Model

In [21]:
tp=0
tr=0
tf=0
ta=0
c=0.
# GENERATING THE CLASSIFICATION REPORT
for text,annot in TEST_DATA:
    doc_to_test=nlp(text)

    d={}
    for ent in doc_to_test.ents:
        d[ent.label_]=[]
    for ent in doc_to_test.ents:
        d[ent.label_].append(ent.text)
   
    for ent in doc_to_test.ents:
        d[ent.label_]=[0,0,0,0,0,0]
    for ent in doc_to_test.ents:
        doc_gold_text= nlp.make_doc(text)
        gold = GoldParse(doc_gold_text, entities=annot.get("entities"))
        y_true = [ent.label_ if ent.label_ in x else 'Not '+ent.label_ for x in gold.ner]
        y_pred = [x.ent_type_ if x.ent_type_ ==ent.label_ else 'Not '+ent.label_ for x in doc_to_test]  
        # print(text,y_pred)
        if(d[ent.label_][0]==0):
            #print("For Entity "+ent.label_+"\n")   
            #f.write(classification_report(y_true, y_pred)+"\n")
            (p,r,f,s)= precision_recall_fscore_support(y_true,y_pred,average='weighted')
            a=accuracy_score(y_true,y_pred)
            d[ent.label_][0]=1
            d[ent.label_][1]+=p
            d[ent.label_][2]+=r
            d[ent.label_][3]+=f
            d[ent.label_][4]+=a
            d[ent.label_][5]+=1
    c+=1
#print(d)
for i in d:
    #print("\n For Entity "+i+"\n")
    print("Accuracy : "+str((d[i][4]/d[i][5])*100)+"%")
    print("Precision : "+str(round(d[i][1]/d[i][5],2)))
    print("Recall : "+str(round(d[i][2]/d[i][5],2)))
    print("F-score : "+str(round(d[i][3]/d[i][5],2)))

Accuracy : 90.0%
Precision : 0.91
Recall : 0.9
F-score : 0.89


## Save Model

In [9]:
output_dir=Path("/Users/zakaria/anaconda3/ML-Material/Integrify/project/sp_zak")
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)#

    # test the saved model
    print("Loading from", output_dir)
    nlp2 = spacy.load(output_dir)
    for text, _ in TEST_DATA:
        doc = nlp2(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Saved model to /Users/zakaria/anaconda3/ML-Material/Integrify/project/sp_zak
Loading from /Users/zakaria/anaconda3/ML-Material/Integrify/project/sp_zak
Entities [('94 cm', 'SIZE')]
Entities []
Entities [('Ziwi Peak', 'BRAND')]
Entities [('Finger', 'BRAND')]
Entities [('4/0', 'SIZE')]
Entities [('Golden', 'BRAND')]
Entities []
Entities [('Ayleen', 'BRAND')]
Entities []
Entities [('170 mm', 'SIZE'), ('20 mm', 'SIZE')]
Entities []
Entities [('Laivastonsininen', 'COLOR')]
Entities [('Headwear', 'COLOR'), ('Sininen', 'COLOR')]
Entities [('Valkoinen', 'COLOR'), ('ASICS SportStyle', 'BRAND')]
Entities []
Entities []
Entities [('Ruskea', 'COLOR'), ('R.M. Williams', 'BRAND')]
Entities [('Vihreä', 'COLOR')]
Entities []
Entities []
Entities [('21', 'SIZE')]
Entities [('CK', 'BRAND')]
Entities [('Nahkiss', 'BRAND')]
Entities []
Entities [('Givenchy', 'BRAND')]
Entities [('Vihreä', 'COLOR')]
Entities []
Entities []
Entities [('Gummy', 'BRAND')]
Entities []
Entities [('FDB', 'BRAND')]
Entities [('Cl

## Test with Test Data

In [10]:
for text, _ in TEST_DATA[0:3]:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('94 cm', 'SIZE')]
Entities []
Entities [('Ziwi Peak', 'BRAND')]


In [13]:
for n in range(0,20):
    doc = nlp(TEST_DATA[n][0])
    spacy.displacy.render(doc, style="ent")